Возьмите регламент по проведению государственной экологической экспертизы  по ссылке https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM и на его основе сделайте базу знаний для ответа на вопросы по этому регламенту.

Напишите промпт самостоятельно. Предусмотрите случаи, когда пользователи будут задавать вопросы, не касающиеся документа - на эти случаи дайте указания модели в промпте. Не забудьте установить настройки доступа на "все, у кого есть ссылка".

 Создайте нейро-консультанта по данной базе знаний. Проверьте работу созданного нейро-консультанта на самостоятельно сформулированных вопросах.

In [ ]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai==1.41.1 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 939.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.

In [ ]:
# запустите эту ячейку, если используете секретный ключ в колабе

from openai import OpenAI
import os
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM') # заполните аргумент функции

In [ ]:
# выведем начало документа
data_from_url[:2000]

Предобрабатываем БЗ таким образом, чтобы можно было разметить ее в формате маркдаун разметки

In [ ]:
def text_to_markdown(text):
    # Добавляем заголовок 1 уровня на основе римских чисел (без переноса строки)
    # и дублируем его строчкой ниже - иначе эта информация перенесется в метаданные, а порой она бывает полезной.
    def replace_header1(match):
        return f"# {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3}|IV|V)\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст, выделенный жирным шрифтом (он заключен между *)
    # и дублируем его строчкой ниже
    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    return text

In [ ]:
markdown = text_to_markdown(data_from_url)
print(markdown[:15000])

In [ ]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/1fm7-syumbz-mIcx-R9QoiEZ3WsWRGapsTAPeR1FzpCM/edit?usp=sharing') # заполните параметр ссылкой на составленный вами промпт. Не забудьте открыть доступ

In [ ]:
print(system[:1000])

In [ ]:
# делим БЗ на чанки при помощи MarkdownHeaderTextSplitter, так как предварительно мы ее разметили именно таким образом
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2")
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
fragments = markdown_splitter.split_text(markdown)

In [ ]:
# выводим несколько чанков, чтобы убедиться, что все получилось в необходимом нам формате
fragments[2:4]

In [ ]:
len(fragments)

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(fragments, embeddings)

In [ ]:
def answer_index(system, topic, search_index, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4-turbo",             # выберите модель самостоятельно
        messages=messages,
        temperature=0.2         # укажите значение
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

Проверьте работу нейро-консультанта на сформулированных вами вопросах (достаточно 1-2)

In [ ]:
# вопрос пользователя
topic= "Сколько дней составляет срок оплаты проведения государственной экологической экспертизы?"                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

In [ ]:
# вопрос пользователя
topic= "Что запрещается требовать от Заявителя?"                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

In [ ]:
# вопрос пользователя
topic= "Какой максимальный срок ожидания в очереди при подаче Заявителем заявления о предоставлении государственной услуги и при получении результата государственной услуги?"                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans